In [1]:
!pip install binance_historical_data mplfinance stable-baselines3 backtesting tensorboard binance ta


[notice] A new release of pip is available: 23.2.1 -> 23.3.2
[notice] To update, run: pip install --upgrade pip


In [2]:
#!git clone https://github.com/Klok-e/python-price-predict-experiment.git

import sys

sys.path.insert(0, 'python-price-predict-experiment')

In [ ]:
import numpy as np
import pandas as pd
from util import download_data, full_preprocess, full_handle_tickers

df_tickers = download_data()
df_tickers = full_handle_tickers(df_tickers)

['ETHBTC', 'LTCBTC', 'BNBBTC', 'NEOBTC', 'QTUMETH', 'EOSETH', 'SNTETH', 'BNTETH', 'BCCBTC', 'GASBTC', 'BNBETH', 'BTCUSDT', 'ETHUSDT', 'HSRBTC', 'OAXETH', 'DNTETH', 'MCOETH', 'ICNETH', 'MCOBTC', 'WTCBTC', 'WTCETH', 'LRCBTC', 'LRCETH', 'QTUMBTC', 'YOYOBTC', 'OMGBTC', 'OMGETH', 'ZRXBTC', 'ZRXETH', 'STRATBTC', 'STRATETH', 'SNGLSBTC', 'SNGLSETH', 'BQXBTC', 'BQXETH', 'KNCBTC', 'KNCETH', 'FUNBTC', 'FUNETH', 'SNMBTC', 'SNMETH', 'NEOETH', 'IOTABTC', 'IOTAETH', 'LINKBTC', 'LINKETH', 'XVGBTC', 'XVGETH', 'SALTBTC', 'SALTETH', 'MDABTC', 'MDAETH', 'MTLBTC', 'MTLETH', 'SUBBTC', 'SUBETH', 'EOSBTC', 'SNTBTC', 'ETCETH', 'ETCBTC', 'MTHBTC', 'MTHETH', 'ENGBTC', 'ENGETH', 'DNTBTC', 'ZECBTC', 'ZECETH', 'BNTBTC', 'ASTBTC', 'ASTETH', 'DASHBTC', 'DASHETH', 'OAXBTC', 'ICNBTC', 'BTGBTC', 'BTGETH', 'EVXBTC', 'EVXETH', 'REQBTC', 'REQETH', 'VIBBTC', 'VIBETH', 'HSRETH', 'TRXBTC', 'TRXETH', 'POWRBTC', 'POWRETH', 'ARKBTC', 'ARKETH', 'YOYOETH', 'XRPBTC', 'XRPETH', 'MODBTC', 'MODETH', 'ENJBTC', 'ENJETH', 'STORJBTC', 'ST

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# 
# import time
# import os
# try:
#   os.makedirs("./drive/MyDrive/colab/model-weights/")
# except:
#   pass
# my_model.save(f"./drive/MyDrive/colab/model-weights/model-weights-{time.time()}.pt")

In [ ]:
#%load_ext tensorboard
#%tensorboard --logdir tensorboard

In [ ]:
from model import LSTMExtractor
from env import CustomEnv
from stable_baselines3.common.vec_env import SubprocVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common.env_util import make_vec_env
import torch

from stable_baselines3.common.env_checker import check_env

env = CustomEnv(df_tickers)
check_env(env)
del env

env = make_vec_env(lambda: CustomEnv(df_tickers), n_envs=10, seed=42, vec_env_cls=SubprocVecEnv)

policy_kvargs = dict(activation_fn=torch.nn.LeakyReLU,
                     features_extractor_class=LSTMExtractor,
                     features_extractor_kwargs=dict(lstm_hidden_size=128, lstm_layers=2),
                     net_arch=[256, 256, 256])

# {'gamma': 0.8, 'ent_coef': 0.02, 'gae_lambda': 0.92}
rl_model = PPO("MultiInputPolicy", env,
               verbose=1,
               tensorboard_log="./tensorboard/",
               ent_coef=0.02,
               gae_lambda=0.92,
               gamma=0.8,
               policy_kwargs=policy_kvargs)
# rl_model.set_parameters("rl-model-best-model/best_model/best_model.zip")
rl_model.policy

In [ ]:
from stable_baselines3.common.callbacks import CheckpointCallback, EvalCallback

checkpoint_callback = CheckpointCallback(
    save_freq=50_000,
    save_path="rl-model-checkpoints/",
    name_prefix="rl_model",
    verbose=1,
    save_vecnormalize=True,
    save_replay_buffer=True
)
eval_env = make_vec_env(lambda: CustomEnv(df_tickers, episode_length=2048), seed=42)
eval_callback = EvalCallback(eval_env, best_model_save_path="rl-model-best-model/best_model",
                             log_path="rl-model-best-model/results", eval_freq=50_000, verbose=1,
                             n_eval_episodes=5)

rl_model.learn(total_timesteps=100000, callback=[checkpoint_callback, eval_callback])

In [ ]:
evaluations = np.load("rl-model-best-model/results/evaluations.npz")
evaluations_df = pd.DataFrame()
evaluations_df["timesteps"] = evaluations["timesteps"]
evaluations_df["results"] = evaluations["results"].mean(axis=1)
evaluations_df

In [ ]:
from util import SKIP_STEPS, calculate_observation
# backtesting

from backtesting import Backtest, Strategy, set_bokeh_output


class NeuralNetStrat(Strategy):
    def __init__(self, broker, data, params):
        super().__init__(broker, data, params)
        self.buy_price = None
        self.expected_gain = None

    def init(self):
        pass

    def next(self):
        if len(self.data) > SKIP_STEPS:
            df = self.data.df.iloc[-SKIP_STEPS:].copy()
            df.drop(columns=["Volume"], inplace=True)

            preprocessed, _ = full_preprocess(df, scaler)
            observation, curr_close, _ = calculate_observation(preprocessed, df, self.buy_price)

            action, _ = rl_model.predict(observation, deterministic=True)
            if self.buy_price is None:
                if action == 1:
                    self.buy()
                    print(f"buy")
                    self.buy_price = curr_close
            else:
                if action == 2:
                    commission = 0.001
                    sell_fee = curr_close * (1 - commission)
                    buy_fee = self.buy_price * (1 + commission)
                    gain_from_trade_fee = (sell_fee - buy_fee) / buy_fee
                    print(f"gain {gain_from_trade_fee}")
                    self.sell()
                    self.buy_price = None


backtest_dataset = dataset[-20000:].copy()
backtest_dataset["Volume"] = df[1023:]["Volume"]
bt = Backtest(backtest_dataset, NeuralNetStrat, commission=.001,
              exclusive_orders=True)
stats = bt.run()
stats

In [ ]:
bt._results._equity_curve.head()

In [ ]:
set_bokeh_output(notebook=False)
bt.plot()